In [1]:
import os 

os.chdir('../')
os.getcwd()

'c:\\Users\\arunk\\FraudDetection'

In [2]:
import os
os.environ['NUMBA_DISABLE_JIT'] = '1'

In [3]:
import os
os.environ['NUMBA_DISABLE_JIT'] = '1'
from fraudDetection.logger import logging
from fraudDetection.exception import FraudDetectionException
from fraudDetection.config.configuration import ConfigurationManager
from fraudDetection.components import DataIngestion, DataValidation, DataTransformation,ModelTrainer
from fraudDetection.entity import DataIngestionArtifact, DataValidationArtifact, DataTransformationArtifact, ModelTrainerArtifact

from fraudDetection.constants import CONFIG_FILE_PATH
config = ConfigurationManager(CONFIG_FILE_PATH)
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion_artifacts = DataIngestion(data_ingestion_config).initiate_data_ingestion()

data_validation_config =config.get_data_validation_config()
data_validation_artifacts =DataValidation(data_ingestion_artifacts,data_validation_config).initiate_data_validation()
data_transformation_config =config.get_data_transformation_config()
data_transformation_obj= DataTransformation(data_validation_artifacts,data_ingestion_artifacts,data_transformation_config)


c:\Users\arunk\FraudDetection\env\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\arunk\FraudDetection\env\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\arunk\FraudDetection\env\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWa

====>>>File already exists


In [4]:
from fraudDetection.utils import save_object, read_yaml, load_data, save_numpy_array_data
from fraudDetection.constants import *
schema_path = data_validation_artifacts.schema_file_path
schema = read_yaml(schema_path)
cat_col = schema[DATA_SCHEMA_CATEGORICAL_COLUMN_KEY]
col = schema[DATA_SCHEMA_COLUMNS_KEY]
num_col = schema[DATA_SCHEMA_NUMERICAL_COLUMN_KEY]
print(col)
col.keys()

{'step': 'int64', 'type': 'object', 'amount': 'float64', 'oldbalanceOrg': 'float64', 'newbalanceOrig': 'float64', 'oldbalanceDest': 'float64', 'newbalanceDest': 'float64', 'isFraud': 'int64'}


dict_keys(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud'])

In [5]:
train_df = load_data(data_ingestion_artifacts.train_file_path,col)
test_df = load_data(data_ingestion_artifacts.test_file_path,col)
target_col_name = schema[DATA_SCHEMA_TARGET_COLUMN_KEY][0]
target_col_name

'isFraud'

In [6]:
input_train_df = train_df.drop(target_col_name,axis=1)
target_col = train_df[target_col_name]
print(input_train_df.shape, target_col.shape)
target_col

(39832, 7) (39832,)


0        0
1        0
2        0
3        0
4        0
        ..
19847    0
19848    0
19849    0
19850    0
19851    0
Name: isFraud, Length: 39832, dtype: int64

In [7]:
input_test_df = test_df.drop(target_col_name,axis=1)
target_col_test = test_df[target_col_name]

In [11]:

from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.feature_selection import SelectPercentile, chi2, SelectFromModel
from sklearn.utils.validation import check_is_fitted, check_X_y
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer

In [9]:

# SimpleImputer(missing_values=np.nan, strategy='mean')

class SMOTENCWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features:list, sampling_strategy:float,target_feature_name:str):
        self.categorical_features = categorical_features
        self.sampling_strategy = sampling_strategy
        self.target_feature_name = target_feature_name
        self.smotenc = SMOTENC(sampling_strategy=sampling_strategy, categorical_features=categorical_features)
        self.target_feature_name = target_feature_name

    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.smotenc.fit_resample(X, y)
        return self

    def transform(self, X, y=None) -> pd.DataFrame:
        df_resampled: pd.DataFrame = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return df_resampled
    
class RandomUnderSamplerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, sampling_strategy,target_feature_name):
        self.sampling_strategy = sampling_strategy
        self.sampler = RandomUnderSampler(sampling_strategy=sampling_strategy)
        self.target_feature_name = target_feature_name
    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.sampler.fit_resample(X.drop(columns=[self.target_feature_name]), X[self.target_feature_name])
        
        return self
    
    def transform(self, X, y=None) -> pd.DataFrame:
        df_resampled: pd.DataFrame = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return self.X_resampled_ , self.y_resampled_
    

sampling_pipeline = Pipeline(steps=[
    ('sampling', SMOTENCWrapper(categorical_features=[1], sampling_strategy=0.1,target_feature_name=target_col_name)),
    ('oversample', RandomUnderSamplerWrapper(sampling_strategy=0.5,target_feature_name=target_col_name))
    ])

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, num_strategy='median', cat_strategy='most_frequent', thresh=None):
        self.num_strategy = num_strategy
        self.cat_strategy = cat_strategy
        self.thresh = thresh
        self.num_imputer = SimpleImputer(strategy=self.num_strategy)
        self.cat_imputer = SimpleImputer(strategy=self.cat_strategy)

    def fit(self, X, y):
        if self.thresh is not None:
            drop_cols = X.columns[X.isnull().mean() > self.thresh]
            X.drop(columns=drop_cols, inplace=True)

        num_cols = X.select_dtypes(include=['float', 'int']).columns
        self.num_imputer.fit(X[num_cols])

        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        self.cat_imputer.fit(X[cat_cols])
        self.y = y
        self.X = X
        return self

    def transform(self, X, y=None):
        if self.thresh is not None:
            drop_cols = X.columns[X.isnull().mean() > self.thresh]
            X.drop(columns=drop_cols, inplace=True)

        num_cols = X.select_dtypes(include=['float', 'int']).columns
        X[num_cols] = self.num_imputer.transform(X[num_cols])

        cat_cols = X.select_dtypes(include=['object', 'category']).columns
        X[cat_cols] = self.cat_imputer.transform(X[cat_cols])


        # return {'X':X.values.reshape(-1,1), 'Y':self.y.values.reshape(1,-1)}
        return X
        # return X

imputer = Pipeline(steps=[("CustomImputer", CustomImputer(num_strategy='median', cat_strategy='most_frequent', thresh=.3))])
# make_pipeline([(CustomImputer(num_strategy='median', cat_strategy='most_frequent', thresh=.3), set_output(transform="pandas"))])
column_name = train_df.columns[:-1]


FeatureExtractor = Pipeline(steps=[
  ('feature_selection', SelectFromModel(ExtraTreesClassifier(n_estimators=50,max_features="log2"),threshold=0.05))
])

# XTrain = df_resampled.iloc[:,:-1]
# yTrain = df_resampled.iloc[:,-1]

# make_column_transformer(
# ...       (StandardScaler(),
# ...        make_column_selector(dtype_include=np.number)),  # rating
# ...       (OneHotEncoder(),
# ...        make_column_selector(dtype_include=object))) 

cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)

cat_preprocessor = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ("selector", SelectPercentile(chi2, percentile=50)),
])

num_preprocessor = Pipeline(steps=[
    ('scaler', RobustScaler())
])

# define the preprocessor pipeline with SMOTENC and RandomOverSampler
preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_selector),
    ('num', num_preprocessor, num_selector)
],remainder="passthrough")

pipe_obj1 = Pipeline(steps=[
    ("transformer", preprocessor),
    ("FeatureExtractor",FeatureExtractor)
])

pipe_obj = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
    ("transformer", preprocessor),
    ("FeatureExtractor", FeatureExtractor)
    ])

# print(train_df.columns)
pipe_obj.fit_transform(input_train_df,target_col)

ValueError: Found input variables with inconsistent numbers of samples: [11940, 39832]

In [14]:
target_feature_name = target_col
import numpy as np

class SMOTENCWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_features:list, sampling_strategy:float):
        self.categorical_features = categorical_features
        self.sampling_strategy = sampling_strategy
        self.smotenc = SMOTENC(sampling_strategy=self.sampling_strategy, categorical_features=self.categorical_features)

    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.smotenc.fit_resample(X, y)
        return self

    def transform(self, X, y=None) -> pd.DataFrame:
        df_resampled: pd.DataFrame = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_)], axis=1)
        return df_resampled
    
class RandomUnderSamplerWrapper(BaseEstimator, TransformerMixin):

    def __init__(self, sampling_strategy):
        self.sampling_strategy = sampling_strategy
        self.sampler = RandomUnderSampler(sampling_strategy=self.sampling_strategy)
    
    def fit(self, X, y):
        self.X_resampled_, self.y_resampled_ = self.sampler.fit_resample(X.iloc[:,:-1], X.iloc[:,-1])
        return self
    
    def transform(self, X, y=None) -> pd.DataFrame:
        # df_resampled: pd.DataFrame = pd.concat([pd.DataFrame(self.X_resampled_), pd.Series(self.y_resampled_, name=self.target_feature_name)], axis=1)
        return self.X_resampled_, self.y_resampled_


class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, num_strategy='median', cat_strategy='most_frequent', thresh=None):
        self.num_strategy = num_strategy
        self.cat_strategy = cat_strategy
        self.thresh = thresh
        self.num_imputer = SimpleImputer(strategy=self.num_strategy)
        self.cat_imputer = SimpleImputer(strategy=self.cat_strategy)

    def fit(self, X, y):
        if self.thresh is not None:
            drop_cols = X.columns[X.isnull().mean() > self.thresh]
            X.drop(columns=drop_cols, inplace=True)

        num_cols = X.select_dtypes(include=np.number).columns
        self.num_imputer.fit(X[num_cols])

        cat_cols = X.select_dtypes(include='O').columns
        self.cat_imputer.fit(X[cat_cols])
        self.y = y
        self.X = X
        return self

    def transform(self, X, y=None):
        if self.thresh is not None:
            drop_cols = X.columns[X.isnull().mean() > self.thresh]
            X.drop(columns=drop_cols, inplace=True)

        num_cols = X.select_dtypes(include=np.number).columns
        X[num_cols] = self.num_imputer.transform(X[num_cols])

        cat_cols = X.select_dtypes(include='O').columns
        X[cat_cols] = self.cat_imputer.transform(X[cat_cols])

        return X
    

imputer = Pipeline([("CustomImputer",CustomImputer(thresh=0.3))])

sampling_pipeline = Pipeline(steps=[
('sampling', SMOTENCWrapper(categorical_features=[1], sampling_strategy=0.1)),
('oversample', RandomUnderSamplerWrapper(sampling_strategy=0.5))
])

cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)
cat_preprocessor = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ("selector", SelectPercentile(chi2, percentile=50)),
])
num_preprocessor = Pipeline(steps=[
    ('scaler', RobustScaler())
])

preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_selector),
    ('num', num_preprocessor, num_selector)
],remainder="passthrough")

feature_extractor = Pipeline(steps=[
    ('feature_selection', SelectFromModel(ExtraTreesClassifier(n_estimators=50,max_features="log2"),threshold=0.01))
])
pipe_obj = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline)
])
pipe_obj1 = Pipeline([
    ("transformer", preprocessor),
    ("FeatureExtractor",feature_extractor)
])
pipe_obj, pipe_obj1

pipe_obj.fit_transform(input_train_df,target_col)


(            step      type         amount  oldbalanceOrg  newbalanceOrig   
 0      12.000000  CASH_OUT   70909.370000   3.307100e+04            0.00  \
 1      13.000000  CASH_OUT  778489.080000   9.540000e+02            0.00   
 2       9.000000   CASH_IN  113915.410000   2.224160e+07     22355513.35   
 3       9.000000  CASH_OUT  306102.950000   2.203776e+04            0.00   
 4      11.000000   PAYMENT   10951.180000   4.280000e+02            0.00   
 ...          ...       ...            ...            ...             ...   
 11935   7.578711  TRANSFER   24218.092797   2.421809e+04            0.00   
 11936   3.064143  TRANSFER   26718.465116   2.671847e+04            0.00   
 11937   3.869439  CASH_OUT  230407.307522   2.304073e+05            0.00   
 11938   8.199136  TRANSFER  763905.336899   7.639053e+05            0.00   
 11939   1.067674  CASH_OUT    3485.137791   3.485138e+03            0.00   
 
        oldbalanceDest  newbalanceDest  
 0        9.849888e+06    1.08640

In [ ]:
imputer = Pipeline([("CustomImputer",CustomImputer(thresh=0.3))])

class PipelineWithY(Pipeline):
    def transform(self, X, y=None):
        Xt = X
        for name, transform in self.steps[:-1]:
            # all transformations except the last one
            if hasattr(transform, 'transform'):
                Xt = transform.transform(Xt)
        # last transformation, handle differently to get X and y
        name, last_transform = self.steps[-1]
        if hasattr(last_transform, 'transform'):
            Xt = last_transform.transform(Xt)
        # return as tuple of (X, y)
        return Xt, y

pipe_obj = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
    # ('target_extractor', FunctionTransformer(lambda df: (df.drop(columns=target_col_name), df[target_col_name]))),
    # ("transformer", preprocessor)
])

pipe_obj.fit(input_train_df,target_col)
df_resampled =pipe_obj.transform(input_train_df)

KeyError: '[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
XTrain = df_resampled.iloc[:,:-1]
yTrain = df_resampled.iloc[:,-1]
print(f"XTrain.shape: {XTrain.shape}, YTrain.shape: {yTrain.shape}" )
yTrain.values.reshape(-1,1)

XTrain.shape: (11940, 7), YTrain.shape: (11940,)


array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int64)

: 

: 

In [ ]:
# define a function to convert the output of pipe_obj into XTrain and yTrain
def split_data(df):
    X_train = df.iloc[:,:-1]
    y_train = df.iloc[:,-1]
    return y_train, X_train
preprocessor = ColumnTransformer([
    ('cat', cat_preprocessor, cat_col),
    ('num', num_preprocessor, num_col)
],remainder="passthrough")

pipe_obj1 = Pipeline(steps=[
    ("transformer", preprocessor),
    ("FeatureExtractor",FeatureExtractor)
])

# define the combined pipeline
pipe_obj = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
])

df_resampled = pipe_obj.fit_transform(input_train_df,target_col)

XTrain = df_resampled.iloc[:,:-1]
yTrain = df_resampled.iloc[:,-1]


train_arr = pipe_obj1.fit_transform(XTrain,yTrain)
col_names =pipe_obj1.get_feature_names_out()
train_df = pd.DataFrame(train_arr,columns=col_names)
print(train_df.shape, yTrain.shape)


train_df_with_y = pd.concat([train_df, yTrain], axis=1)
print(train_df_with_y.shape)
train_df_with_y

pipe_obj2 = Pipeline([
    ("transformer",preprocessor)
])

pipe_obj2.fit_transform(input_test_df,target_col_test)
pipe_obj2.get_feature_names_out()

c:\Users\arunk\FraudDetection\env\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


(11940, 8) (11940,)
(11940, 9)


array(['cat__type_PAYMENT', 'cat__type_TRANSFER', 'num__step',
       'num__amount', 'num__oldbalanceOrg', 'num__newbalanceOrig',
       'num__oldbalanceDest', 'num__newbalanceDest'], dtype=object)

: 

: 

In [ ]:
pipe_obj4 = Pipeline([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
     ("transformer", preprocessor),
    ("FeatureExtractor",FeatureExtractor)
])
pipe_obj4[-2:]


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('selector',
                                                                   SelectPercentile(percentile=50,
                                                                                    score_func=<function chi2 at 0x0000023260D850D8>))]),
                                                  BoxList(['type'])),
                                                 ('num',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  BoxList(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']))])),
                ('FeatureExtractor',
                 Pipeline(steps=[('feature_selection',
                                  SelectFromModel(estimator=ExtraTreesClassifier(max_features='log2',
                                                                                 n_estimators=50),
                                                  threshold=0.05))]))])

: 

: 

In [ ]:
import numpy as np
X = np.random.random_sample((3,2))*10
print(X)
print(X[:,0])
print(X[:,0].shape)
print(X[:,0].reshape(-1,1))

[[3.8159798  7.80384405]
 [6.24182374 8.99134049]
 [8.11934961 7.00098384]]
[3.8159798  6.24182374 8.11934961]
(3,)
[[3.8159798 ]
 [6.24182374]
 [8.11934961]]


: 

: 

In [ ]:
df.columns.groupby(df.dtypes)
df.columns.get_loc("type")

NameError: name 'df' is not defined

: 

: 

In [ ]:

cat_col
num_col
target_col_name

from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
import numpy as np
cat_selector = make_column_selector(dtype_include=object)
num_selector =make_column_selector(dtype_include=np.number)

cat_tree_processor = OneHotEncoder()
num_tree_processor = SimpleImputer(strategy='median')

tree_processor = make_column_transformer(
    (cat_preprocessor,cat_selector),
    (num_preprocessor,num_selector))

cat_linear_processor = OneHotEncoder(handle_unknown="ignore")
num_linear_processor = make_pipeline(
    RobustScaler(),SimpleImputer(strategy='median')
)
linear_preprocessor = make_column_transformer(
    (num_linear_processor,num_selector), (cat_linear_processor,cat_selector)
    )

: 

: 

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_thresh, upper_thresh):
        super().__init__()
        self.lower_thresh = lower_thresh
        self.upper_thresh = upper_thresh
 
    def fit(self, X, y):
        null_prop_ = X.isnull().sum()/len(X)
        drop_cols= null_prop_[null_prop_ > self.upper_thresh].index 
        X.drop(columns=drop_cols,inplace= True)

        drop_rows = X.loc[X.isnull().mean(axis=1).between(self.lower_thresh,self.upper_thresh)].index
        X.drop(index=drop_rows, inplace=True)
        self.y =y
        
        return self

    def transform(self, X, y=None):
        # fill Age
        X.fillna(X.median(),inplace=True)
        
        return X,self.y

# pipe_obj = Pipeline([
#     ("CustomImputer",imputer),
#     ("sampling_pipeline", sampling_pipeline),
#     # ('target_extractor', FunctionTransformer(lambda df: ( df[target_col_name], df.drop(columns=target_col_name)))),
#     ("ta",preprocessor)
# ])

class PipelineWithY(Pipeline):
    def transform(self, X, y=None):
        Xt = X
        for name, transform in self.steps[:-1]:
            # all transformations except the last one
            if hasattr(transform, 'transform'):
                Xt = transform.transform(Xt)
        # last transformation, handle differently to get X and y
        name, last_transform = self.steps[-1]
        if hasattr(last_transform, 'transform'):
            Xt = last_transform.transform(Xt)
        # return as tuple of (X, y)
        return Xt, y
    
pipe_obj = PipelineWithY([
    ("CustomImputer",imputer),
    ("sampling_pipeline", sampling_pipeline),
    ('target_extractor', FunctionTransformer(lambda df: ( df[target_col_name], df.drop(columns=target_col_name)))),
    ("ta",preprocessor)
])
pipe_obj.fit(input_train_df,target_col)
pipe_obj.transform(input_train_df)

NameError: name 'Pipeline' is not defined

: 

: 

In [ ]:
enc = OneHotEncoder()
enc_data = pd.DataFrame(enc.fit_transform(
    input_train_df[['type']]).toarray())
  
# Merge with main
New_df = input_train_df.join(enc_data)
New_df.drop(columns=['type'],inplace=True)
names = enc.get_feature_names_out()
New_df.select_dtypes(include='integer').columns

# OH_X_train= pd.concat([New_df,enc_data], axis=1)
# OH_X_train

Index(['step'], dtype='object')

: 

: 

In [ ]:
from sklearn.utils.validation import check_is_fitted, check_X_y
class GroupImputer(BaseEstimator, TransformerMixin):
    '''
    Class used for imputing missing values in a pd.DataFrame using either mean or median of a group.
    
    Parameters
    ----------    
    group_cols : list
        List of columns used for calculating the aggregated value 
    target : str
        The name of the column to impute
    metric : str
        The metric to be used for remplacement, can be one of ['mean', 'median']
    Returns
    -------
    X : array-like
        The array with imputed values in the target column
    '''
    def __init__(self, group_cols, target, metric='mean'):
        
        assert metric in ['mean', 'median'], 'Unrecognized value for metric, should be mean/median'
        assert type(group_cols) == list, 'group_cols should be a list of columns'
        assert type(target) == str, 'target should be a string'
        
        self.group_cols = group_cols
        self.target = target
        self.metric = metric
    
    def fit(self, X, y=None):
        
        assert pd.isnull(X[self.group_cols]).any(axis=None) == False, 'There are missing values in group_cols'
        
        impute_map = X.groupby(self.group_cols)[self.target].agg(self.metric) \
                                                            .reset_index(drop=False)
        
        self.impute_map_ = impute_map
        
        return self 
    
    def transform(self, X, y=None):
        
        # make sure that the imputer was fitted
        check_is_fitted(self, 'impute_map_')
        
        X = X.copy()
        
        for index, row in self.impute_map_.iterrows():
            ind = (X[self.group_cols] == row[self.group_cols]).all(axis=1)
            X.loc[ind, self.target] = X.loc[ind, self.target].fillna(row[self.target])
        
        return X.values
    
GroupImputer(group_cols, target, metric='median')

  # weights = {0: n_samples / (2 * np.bincount(y_train_c)[0]), 1: n_samples / (2 * np.bincount(y_train_c)[1])}

: 

: 